In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
lista_imagens = sorted(os.listdir('./imagens_alpha/'))
num_imag_alpha = 0
num_imag_s_alpha = 0
print (lista_imagens)

['test_001.png', 'test_002.png', 'test_003.png', 'test_004.png', 'test_005.png', 'test_006.png', 'test_007.png', 'test_008.png', 'test_009.png', 'test_010.png', 'test_011.png', 'test_012.png', 'test_013.png', 'test_014.png', 'test_015.png', 'test_016.png', 'test_017.png', 'test_018.png', 'test_019.png', 'test_020.png', 'test_021.png', 'test_022.png', 'test_023.png', 'test_024.png', 'test_025.png', 'test_026.png', 'test_027.png', 'test_028.png', 'test_029.png', 'test_030.png', 'test_031.png', 'test_032.png', 'test_033.png', 'test_034.png', 'test_035.png', 'test_036.png', 'test_037.png', 'test_038.png', 'test_039.png', 'test_040.png', 'test_041.png', 'test_042.png', 'test_043.png', 'test_044.png', 'test_045.png', 'test_046.png', 'test_047.png', 'test_048.png', 'test_049.png', 'test_050.png', 'test_051.png', 'test_052.png', 'test_053.png', 'test_054.png', 'test_055.png', 'test_056.png', 'test_057.png', 'test_058.png', 'test_059.png', 'test_060.png', 'test_061.png', 'test_062.png', 'test_0

In [3]:
for imagem_entry in lista_imagens:
    img = cv2.imread("./imagens_alpha/" + imagem_entry, cv2.IMREAD_UNCHANGED)
    altura,largura,canais = img.shape
    lado_da_regiao = 16
    boxes_altura = altura//lado_da_regiao
    boxes_largura = largura//lado_da_regiao
    #print(boxes_altura)
    #print(boxes_largura)
    for i in range(1,boxes_altura):
        for j in range(1,boxes_largura):
            middle_altura = i*lado_da_regiao
            middle_largura = j*lado_da_regiao
            lado_div_2 = lado_da_regiao//2
            roi = img[middle_altura-lado_div_2:middle_altura+lado_div_2,middle_largura-lado_div_2:middle_largura+lado_div_2]
            #cv2.imshow('imagem', roi)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
            tem_alpha = False
            for altura in range(lado_da_regiao):
                for largura in range(lado_da_regiao):
                    alpha = roi.item(altura,largura,3)
                    r = roi.item(altura,largura,0)
                    if(alpha != 255):
                        tem_alpha = True
            if (tem_alpha):
                num_imag_alpha += 1
                cv2.imwrite('recorte_placas/'+str(num_imag_alpha)+'.jpg',roi)
            else:
                num_imag_s_alpha += 1
                cv2.imwrite('recorte_sem_placa/'+str(num_imag_s_alpha)+'.jpg',roi)

In [3]:
import re
import math

from skimage import feature
from imutils import paths
def label(n):
    return {
        0: 'c1',
        1: 'c2',
    }[n]

In [4]:
lista_imagens = sorted(os.listdir('./recorte_placas/'))
X = []
for imagem_entry in lista_imagens:
    imagem = cv2.imread('./recorte_placas/'+imagem_entry)
    altura, largura, _ = imagem.shape
    cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    rgb   = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        
    r_histograma = cv2.calcHist([rgb], [0], None, [256], [0, 256])/(altura*largura)
    g_histograma = cv2.calcHist([rgb], [1], None, [256], [0, 256])/(altura*largura)
    b_histograma = cv2.calcHist([rgb], [2], None, [256], [0, 256])/(altura*largura)
    
    lbp = feature.local_binary_pattern(cinza, 59, 1, method="uniform")
    (lbp_histograma, _) = np.histogram(lbp.ravel(), bins=59, range=(0, 59))
    lbp_histograma = lbp_histograma.astype("float")
    lbp_histograma /= (lbp_histograma.sum())

    X_image = [r_histograma, g_histograma, b_histograma, lbp_histograma]
    
    X_image_aux = []
    for aux in X_image:
        X_image_aux = np.append(X_image_aux, np.ravel(aux))
    
    X_image = [i for i in X_image_aux]
    X_image.append(label(0))
    X.append(X_image)

In [5]:
lista_imagens = sorted(os.listdir('./recorte_sem_placa/'))
Y = []
for imagem_entry in lista_imagens:
    imagem = cv2.imread('./recorte_sem_placa/'+imagem_entry)
    altura, largura, _ = imagem.shape
    cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    rgb   = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        
    r_histograma = cv2.calcHist([rgb], [0], None, [256], [0, 256])/(altura*largura)
    g_histograma = cv2.calcHist([rgb], [1], None, [256], [0, 256])/(altura*largura)
    b_histograma = cv2.calcHist([rgb], [2], None, [256], [0, 256])/(altura*largura)
    
    lbp = feature.local_binary_pattern(cinza, 59, 1, method="uniform")
    (lbp_histograma, _) = np.histogram(lbp.ravel(), bins=59, range=(0, 59))
    lbp_histograma = lbp_histograma.astype("float")
    lbp_histograma /= (lbp_histograma.sum())

    X_image = [r_histograma, g_histograma, b_histograma, lbp_histograma]
    
    X_image_aux = []
    for aux in X_image:
        X_image_aux = np.append(X_image_aux, np.ravel(aux))
    
    X_image = [i for i in X_image_aux]
    X_image.append(label(1))
    Y.append(X_image)
df = pd.DataFrame(X)
df.to_csv('feat.csv', header=False, index=False)